#Transfer Learning

In practice, very few people train an entire Convolutional Network from scratch (with random initialization), because it is relatively rare to have a dataset of sufficient size. Instead, it is common to pretrain a ConvNet on a very large dataset (e.g. ImageNet, which contains 1.2 million images with 1000 categories), and then use the ConvNet either as an initialization or a fixed feature extractor for the task of interest. The three major Transfer Learning scenarios look as follows:

ConvNet as fixed feature extractor. Take a ConvNet pretrained on ImageNet, remove the last fully-connected layer (this layer’s outputs are the 1000 class scores for a different task like ImageNet), then treat the rest of the ConvNet as a fixed feature extractor for the new dataset. In an AlexNet, this would compute a 4096-D vector for every image that contains the activations of the hidden layer immediately before the classifier. We call these features CNN codes. It is important for performance that these codes are ReLUd (i.e. thresholded at zero) if they were also thresholded during the training of the ConvNet on ImageNet (as is usually the case). Once you extract the 4096-D codes for all images, train a linear classifier (e.g. Linear SVM or Softmax classifier) for the new dataset.

#Fine-tuning the ConvNet. 

The second strategy is to not only replace and retrain the classifier on top of the ConvNet on the new dataset, but to also fine-tune the weights of the pretrained network by continuing the backpropagation. It is possible to fine-tune all the layers of the ConvNet, or it’s possible to keep some of the earlier layers fixed (due to overfitting concerns) and only fine-tune some higher-level portion of the network. This is motivated by the observation that the earlier features of a ConvNet contain more generic features (e.g. edge detectors or color blob detectors) that should be useful to many tasks, but later layers of the ConvNet becomes progressively more specific to the details of the classes contained in the original dataset. In case of ImageNet for example, which contains many dog breeds, a significant portion of the representational power of the ConvNet may be devoted to features that are specific to differentiating between dog breeds.

#Pretrained models. 

Since modern ConvNets take 2-3 weeks to train across multiple GPUs on ImageNet, it is common to see people release their final ConvNet checkpoints for the benefit of others who can use the networks for fine-tuning. For example, the Caffe library has a Model Zoo where people share their network weights.
When and how to fine-tune? How do you decide what type of transfer learning you should perform on a new dataset? This is a function of several factors, but the two most important ones are the size of the new dataset (small or big), and its similarity to the original dataset (e.g. ImageNet-like in terms of the content of images and the classes, or very different, such as microscope images). Keeping in mind that ConvNet features are more generic in early layers and more original-dataset-specific in later layers, here are some common rules of thumb for navigating the 4 major scenarios:

New dataset is small and similar to original dataset. Since the data is small, it is not a good idea to fine-tune the ConvNet due to overfitting concerns. Since the data is similar to the original data, we expect higher-level features in the ConvNet to be relevant to this dataset as well. Hence, the best idea might be to train a linear classifier on the CNN codes.
New dataset is large and similar to the original dataset. Since we have more data, we can have more confidence that we won’t overfit if we were to try to fine-tune through the full network.
New dataset is small but very different from the original dataset. Since the data is small, it is likely best to only train a linear classifier. Since the dataset is very different, it might not be best to train the classifier form the top of the network, which contains more dataset-specific features. Instead, it might work better to train the SVM classifier from activations somewhere earlier in the network.
New dataset is large and very different from the original dataset. Since the dataset is very large, we may expect that we can afford to train a ConvNet from scratch. However, in practice it is very often still beneficial to initialize with weights from a pretrained model. In this case, we would have enough data and confidence to fine-tune through the entire network.
Practical advice. There are a few additional things to keep in mind when performing Transfer Learning:

Constraints from pretrained models. Note that if you wish to use a pretrained network, you may be slightly constrained in terms of the architecture you can use for your new dataset. For example, you can’t arbitrarily take out Conv layers from the pretrained network. However, some changes are straight-forward: Due to parameter sharing, you can easily run a pretrained network on images of different spatial size. This is clearly evident in the case of Conv/Pool layers because their forward function is independent of the input volume spatial size (as long as the strides “fit”). In case of FC layers, this still holds true because FC layers can be converted to a Convolutional Layer: For example, in an AlexNet, the final pooling volume before the first FC layer is of size [6x6x512]. Therefore, the FC layer looking at this volume is equivalent to having a Convolutional Layer that has receptive field size 6x6, and is applied with padding of 0.

Learning rates. It’s common to use a smaller learning rate for ConvNet weights that are being fine-tuned, in comparison to the (randomly-initialized) weights for the new linear classifier that computes the class scores of your new dataset. This is because we expect that the ConvNet weights are relatively good, so we don’t wish to distort them too quickly and too much (especially while the new Linear Classifier above them is being trained from random initialization)

#ImageNet 

The id’s of the 2 object classes we are considering are given below

```
Ships and bikes
```

In [0]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import cv2
import PIL.Image
import urllib
page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n04194289")#ship synset
print(page.content)
# BeautifulSoup is an HTML parsing library
soup = BeautifulSoup(page.content, 'html.parser')#puts the content of the website into the soup variable, each url on a different line

bikes_page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n02834778")#bicycle synset
print(bikes_page.content)
# BeautifulSoup is an HTML parsing library
from bs4 import BeautifulSoup
bikes_soup = BeautifulSoup(bikes_page.content, 'html.parser')#puts the content of the website into the soup variable, each url on a different line

str_soup=str(soup)#convert soup to string so it can be split
type(str_soup)
split_urls=str_soup.split('\r\n')#split so each url is a different possition on a list
print(len(split_urls))#print the length of the list so you know how many urls you have

bikes_str_soup=str(bikes_soup)#convert soup to string so it can be split
type(bikes_str_soup)
bikes_split_urls=bikes_str_soup.split('\r\n')#split so each url is a different possition on a list
print(len(bikes_split_urls))

!mkdir /content/train #create the Train folder
!mkdir /content/train/ships #create the ships folder
!mkdir /content/train/bikes #create the bikes folder
!mkdir /content/validation
!mkdir /content/validation/ships #create the ships folder
!mkdir /content/validation/bikes #create the bikes folder

#code part 4
img_rows, img_cols = 32, 32 #number of rows and columns to convert the images to
input_shape = (img_rows, img_cols, 3)#format to store the images (rows, columns,channels) called channels last

def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
	resp = urllib.request.urlopen(url)
	image = np.asarray(bytearray(resp.read()), dtype="uint8")
	image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
	# return the image
	return image

n_of_training_images=100#the number of training images to use
for progress in range(n_of_training_images):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not split_urls[progress] == None:
      try:
        I = url_to_image(split_urls[progress])
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/train/ships/img'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None

#do the same for bikes:
for progress in range(n_of_training_images):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not bikes_split_urls[progress] == None:
      try:
        I = url_to_image(bikes_split_urls[progress])
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/train/bikes/img'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None
        
        
#Validation data:

for progress in range(50):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not split_urls[progress] == None:
      try:
        I = url_to_image(split_urls[n_of_training_images+progress])#get images that are different from the ones used for training
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/validation/ships/img'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None

#do the same for bikes:
for progress in range(50):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not bikes_split_urls[progress] == None:
      try:
        I = url_to_image(bikes_split_urls[n_of_training_images+progress])#get images that are different from the ones used for training
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/validation/bikes/img'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None
        
print("\nTRAIN:\n")          
print("\nlist the files inside ships directory:\n")        
!ls /content/train/ships #list the files inside ships
print("\nlist the files inside bikes directory:\n")
!ls /content/train/bikes #list the files inside bikes
print("\nVALIDATION:\n")
print("\nlist the files inside ships directory:\n")        
!ls /content/validation/ships #list the files inside ships
print("\nlist the files inside bikes directory:\n")
!ls /content/validation/bikes #list the files inside bikes   

#Data pre-processing and data augmentation

In order to make the most of our few training examples, we will "augment" them via a number of random transformations, so that our model would never see twice the exact same picture. This helps prevent overfitting and helps the model generalize better.

In Keras this can be done via the keras.preprocessing.image.ImageDataGenerator class. This class allows you to:

1. configure random transformations and normalization operations to be done on your image data during training
2. instantiate generators of augmented image batches (and their labels) via .flow(data, labels) or .flow_from_directory(directory). These generators can then be used with the Keras model methods that accept data generators as inputs, fit_generator, evaluate_generator and predict_generator.

#Load the pre-trained model VGG16

Our strategy will be as follow: we will only instantiate the convolutional part of the model, everything up to the fully-connected layers. We will then run this model on our training and validation data once, recording the output (the "bottleneck features" from th VGG16 model: the last activation maps before the fully-connected layers) in two numpy arrays. Then we will train a small fully-connected model on top of the stored features.

![alt text](http://upscfever.com/upsc-fever/en/data/images/vgg16_original.png)

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'vgg16_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('vgg16_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('vgg16_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('vgg16_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('vgg16_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

Using TensorFlow backend.


58892288/58889256 [==============================] - 2s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 5ms/step - loss: 2.0958 - accuracy: 0.5833 - val_loss: 1.1852 - val_accuracy: 0.4667
Epoch 2/50
60/60 [==============================] - 0s 930us/step - loss: 1.2055 - accuracy: 0.5833 - val_loss: 1.0890 - val_accuracy: 0.5000
Epoch 3/50
60/60 [==============================] - 0s 1ms/step - loss: 0.9813 - accuracy: 0.7333 - val_loss: 2.3753 - val_accuracy: 0.4667
Epoch 4/50
60/60 [==============================] - 0s 939us/step - loss: 0.7364 - accuracy: 0.7333 - val_loss: 1.3707 - val_accuracy: 0.4667
Epoch 5/50
60/60 [==============================] - 0s 945us/step - loss: 0.3679 - accuracy: 0.8167 - val_loss: 1.4402 - val_accuracy: 0.5000
Epoch 6/50
60/60 [==============================] - 0s 973us/step - loss: 0.3304 - accuracy: 0.8667 - va

#Load the pre-trained model ResNet 50

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'resnet50_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.resnet50.ResNet50(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('resnet50_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('resnet50_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('resnet50_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('resnet50_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 5ms/step - loss: 5.7385 - accuracy: 0.4000 - val_loss: 3.2502 - val_accuracy: 0.5000
Epoch 2/50
60/60 [==============================] - 0s 3ms/step - loss: 4.1570 - accuracy: 0.4667 - val_loss: 1.1146 - val_accuracy: 0.5333
Epoch 3/50
60/60 [==============================] - 0s 3ms/step - loss: 2.1654 - accuracy: 0.5833 - val_loss: 1.4135 - val_accuracy: 0.5000
Epoch 4/50
60/60 [==============================] - 0s 3ms/step - loss: 1.3191 - accuracy: 0.6000 - val_loss: 1.1553 - val_accuracy: 0.5000
Epoch 5/50
60/60 [==============================] - 0s 3ms/step - loss: 1.0992 - accuracy: 0.6333 - val_loss: 1.1308 - val_accuracy: 0.5000
Epoch 6/50
60/60 [==============================] - 0s 3ms/step - loss: 0.7253 - accuracy: 0.6000 - val_loss: 

#Load Xception pretrained on ImageNet

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'xcept_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.xception.Xception(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('xcept_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('xcept_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('xcept_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('xcept_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

83689472/83683744 [==============================] - 2s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 5ms/step - loss: 16.8232 - accuracy: 0.4500 - val_loss: 8.4224 - val_accuracy: 0.5000
Epoch 2/50
60/60 [==============================] - 0s 3ms/step - loss: 7.5468 - accuracy: 0.6333 - val_loss: 7.3415 - val_accuracy: 0.5333
Epoch 3/50
60/60 [==============================] - 0s 3ms/step - loss: 6.8950 - accuracy: 0.7167 - val_loss: 7.5906 - val_accuracy: 0.5000
Epoch 4/50
60/60 [==============================] - 0s 3ms/step - loss: 0.5380 - accuracy: 0.8833 - val_loss: 19.8945 - val_accuracy: 0.5333
Epoch 5/50
60/60 [==============================] - 0s 3ms/step - loss: 1.7488 - accuracy: 0.8500 - val_loss: 9.0996 - val_accuracy: 0.5333
Epoch 6/50
60/60 [==============================] - 0s 3ms/step - loss: 1.6078 - accuracy: 0.9000 - val_loss

#Load pretrained VGG19 

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'vgg19_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.vgg19.VGG19(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('vgg19_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('vgg19_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('vgg19_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('vgg19_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

80142336/80134624 [==============================] - 2s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 3ms/step - loss: 2.2587 - accuracy: 0.4833 - val_loss: 1.2777 - val_accuracy: 0.5000
Epoch 2/50
60/60 [==============================] - 0s 969us/step - loss: 0.9792 - accuracy: 0.6667 - val_loss: 2.8381 - val_accuracy: 0.5000
Epoch 3/50
60/60 [==============================] - 0s 979us/step - loss: 0.8907 - accuracy: 0.6333 - val_loss: 1.3360 - val_accuracy: 0.4000
Epoch 4/50
60/60 [==============================] - 0s 958us/step - loss: 0.9687 - accuracy: 0.6500 - val_loss: 1.1575 - val_accuracy: 0.4667
Epoch 5/50
60/60 [==============================] - 0s 970us/step - loss: 0.1932 - accuracy: 0.9000 - val_loss: 1.1680 - val_accuracy: 0.4667
Epoch 6/50
60/60 [==============================] - 0s 985us/step - loss: 0.3249 - accuracy: 0.8667 - 

#Load pretrained ResNet101

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'ResNet101_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.resnet.ResNet101(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('ResNet101_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('ResNet101_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('ResNet101_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('ResNet101_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

171450368/171446536 [==============================] - 4s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 5ms/step - loss: 3.1464 - accuracy: 0.5833 - val_loss: 2.1300 - val_accuracy: 0.5000
Epoch 2/50
60/60 [==============================] - 0s 3ms/step - loss: 3.0861 - accuracy: 0.4000 - val_loss: 1.3846 - val_accuracy: 0.5000
Epoch 3/50
60/60 [==============================] - 0s 3ms/step - loss: 2.1065 - accuracy: 0.4500 - val_loss: 0.8442 - val_accuracy: 0.4667
Epoch 4/50
60/60 [==============================] - 0s 3ms/step - loss: 1.0268 - accuracy: 0.4833 - val_loss: 1.3668 - val_accuracy: 0.5000
Epoch 5/50
60/60 [==============================] - 0s 3ms/step - loss: 0.8510 - accuracy: 0.5500 - val_loss: 0.7557 - val_accuracy: 0.3667
Epoch 6/50
60/60 [==============================] - 0s 3ms/step - loss: 0.7684 - accuracy: 0.6000 - val_loss

#Load pretrained ResNet152

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'ResNet152_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.resnet.ResNet101(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('ResNet152_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('ResNet152_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('ResNet152_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('ResNet152_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 5ms/step - loss: 3.5222 - accuracy: 0.4833 - val_loss: 4.0025 - val_accuracy: 0.5000
Epoch 2/50
60/60 [==============================] - 0s 3ms/step - loss: 3.7301 - accuracy: 0.5000 - val_loss: 3.0554 - val_accuracy: 0.5000
Epoch 3/50
60/60 [==============================] - 0s 3ms/step - loss: 3.2493 - accuracy: 0.4667 - val_loss: 2.5853 - val_accuracy: 0.5000
Epoch 4/50
60/60 [==============================] - 0s 3ms/step - loss: 1.5511 - accuracy: 0.5500 - val_loss: 1.8684 - val_accuracy: 0.5000
Epoch 5/50
60/60 [==============================] - 0s 3ms/step - loss: 1.3781 - accuracy: 0.4833 - val_loss: 0.8402 - val_accuracy: 0.5000
Epoch 6/50
60/60 [==============================] - 0s 3ms/step - loss: 1.1525 - accuracy: 0.4833 - val_loss: 0.7611 - val_accuracy: 0.5333
Epoch 7/50
60/60 [=================

#Load pretrained ResNet50V2

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'ResNet50V2_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.resnet_v2.ResNet50V2(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('ResNet50V2_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('ResNet50V2_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('ResNet50V2_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('ResNet50V2_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

94674944/94668760 [==============================] - 2s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 5ms/step - loss: 23.9481 - accuracy: 0.5500 - val_loss: 16.5671 - val_accuracy: 0.5333
Epoch 2/50
60/60 [==============================] - 0s 3ms/step - loss: 7.2452 - accuracy: 0.8000 - val_loss: 10.8218 - val_accuracy: 0.4000
Epoch 3/50
60/60 [==============================] - 0s 3ms/step - loss: 2.4653 - accuracy: 0.8500 - val_loss: 10.7405 - val_accuracy: 0.3333
Epoch 4/50
60/60 [==============================] - 0s 3ms/step - loss: 1.7924 - accuracy: 0.8833 - val_loss: 11.1128 - val_accuracy: 0.3667
Epoch 5/50
60/60 [==============================] - 0s 3ms/step - loss: 1.8444 - accuracy: 0.9167 - val_loss: 12.2987 - val_accuracy: 0.4000
Epoch 6/50
60/60 [==============================] - 0s 3ms/step - loss: 0.0567 - accuracy: 0.9667 - val_

#Load pretrained ResNet101V2

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'ResNet101V2_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.resnet_v2.ResNet101V2(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('ResNet101V2_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('ResNet101V2_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('ResNet101V2_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('ResNet101V2_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

171319296/171317808 [==============================] - 4s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 5ms/step - loss: 30.4922 - accuracy: 0.3667 - val_loss: 10.7636 - val_accuracy: 0.4333
Epoch 2/50
60/60 [==============================] - 0s 3ms/step - loss: 5.7027 - accuracy: 0.7667 - val_loss: 9.0807 - val_accuracy: 0.3333
Epoch 3/50
60/60 [==============================] - 0s 3ms/step - loss: 2.1783 - accuracy: 0.8333 - val_loss: 8.9101 - val_accuracy: 0.3333
Epoch 4/50
60/60 [==============================] - 0s 3ms/step - loss: 1.1593 - accuracy: 0.8833 - val_loss: 13.6244 - val_accuracy: 0.3000
Epoch 5/50
60/60 [==============================] - 0s 3ms/step - loss: 0.2963 - accuracy: 0.9667 - val_loss: 12.3554 - val_accuracy: 0.4000
Epoch 6/50
60/60 [==============================] - 0s 3ms/step - loss: 0.7898 - accuracy: 0.9500 - val_

#Load pretrained ResNet152V2

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'ResNet152V2_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.resnet_v2.ResNet152V2(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('ResNet152V2_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('ResNet152V2_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('ResNet152V2_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('ResNet152V2_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

234553344/234545216 [==============================] - 5s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 5ms/step - loss: 22.3311 - accuracy: 0.5167 - val_loss: 13.9392 - val_accuracy: 0.4000
Epoch 2/50
60/60 [==============================] - 0s 3ms/step - loss: 4.5229 - accuracy: 0.7833 - val_loss: 9.8014 - val_accuracy: 0.4333
Epoch 3/50
60/60 [==============================] - 0s 3ms/step - loss: 2.5438 - accuracy: 0.8500 - val_loss: 19.7848 - val_accuracy: 0.3333
Epoch 4/50
60/60 [==============================] - 0s 3ms/step - loss: 1.0229 - accuracy: 0.9167 - val_loss: 18.6276 - val_accuracy: 0.3000
Epoch 5/50
60/60 [==============================] - 0s 3ms/step - loss: 1.3014 - accuracy: 0.9167 - val_loss: 22.1667 - val_accuracy: 0.4000
Epoch 6/50
60/60 [==============================] - 0s 3ms/step - loss: 0.4844 - accuracy: 0.9500 - val

#Load pretrained InceptionV3

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'InceptionV3_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('InceptionV3_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('InceptionV3_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('InceptionV3_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('InceptionV3_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

87916544/87910968 [==============================] - 2s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 4ms/step - loss: 27.1500 - accuracy: 0.4500 - val_loss: 18.1111 - val_accuracy: 0.5000
Epoch 2/50
60/60 [==============================] - 0s 2ms/step - loss: 25.1997 - accuracy: 0.4833 - val_loss: 22.4643 - val_accuracy: 0.5000
Epoch 3/50
60/60 [==============================] - 0s 1ms/step - loss: 10.5553 - accuracy: 0.7167 - val_loss: 11.3271 - val_accuracy: 0.6000
Epoch 4/50
60/60 [==============================] - 0s 1ms/step - loss: 7.6588 - accuracy: 0.7667 - val_loss: 41.4603 - val_accuracy: 0.5000
Epoch 5/50
60/60 [==============================] - 0s 1ms/step - loss: 4.0044 - accuracy: 0.9000 - val_loss: 9.0707 - val_accuracy: 0.5000
Epoch 6/50
60/60 [==============================] - 0s 1ms/step - loss: 3.1483 - accuracy: 0.8000 - val

#Load pretrained InceptionResNetV2

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'InceptionResNetV2_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('InceptionResNetV2_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('InceptionResNetV2_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('InceptionResNetV2_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('InceptionResNetV2_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

219062272/219055592 [==============================] - 5s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 4ms/step - loss: 27.3640 - accuracy: 0.5167 - val_loss: 17.3882 - val_accuracy: 0.5000
Epoch 2/50
60/60 [==============================] - 0s 1ms/step - loss: 11.2883 - accuracy: 0.5667 - val_loss: 6.2828 - val_accuracy: 0.5000
Epoch 3/50
60/60 [==============================] - 0s 1ms/step - loss: 7.3338 - accuracy: 0.6500 - val_loss: 5.6880 - val_accuracy: 0.4667
Epoch 4/50
60/60 [==============================] - 0s 1ms/step - loss: 2.7742 - accuracy: 0.7667 - val_loss: 7.1886 - val_accuracy: 0.5333
Epoch 5/50
60/60 [==============================] - 0s 1ms/step - loss: 2.2596 - accuracy: 0.7833 - val_loss: 10.3757 - val_accuracy: 0.5333
Epoch 6/50
60/60 [==============================] - 0s 1ms/step - loss: 1.5616 - accuracy: 0.8500 - val_

#Load pretrained MobileNet

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'MobileNet_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.mobilenet.MobileNet(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('MobileNet_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('MobileNet_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('MobileNet_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('MobileNet_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


17227776/17225924 [==============================] - 1s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 3ms/step - loss: 22.0346 - accuracy: 0.4667 - val_loss: 4.1174 - val_accuracy: 0.5333
Epoch 2/50
60/60 [==============================] - 0s 2ms/step - loss: 3.6127 - accuracy: 0.7167 - val_loss: 5.4578 - val_accuracy: 0.3667
Epoch 3/50
60/60 [==============================] - 0s 1ms/step - loss: 1.9868 - accuracy: 0.8500 - val_loss: 12.0019 - val_accuracy: 0.5333
Epoch 4/50
60/60 [==============================] - 0s 1ms/step - loss: 1.4809 - accuracy: 0.9000 - val_loss: 6.6601 - val_accuracy: 0.5000
Epoch 5/50
60/60 [==============================] - 0s 1ms/step - loss: 1.9151 - accuracy: 0.8500 - val_loss: 6.7543 - val_accuracy: 0.4333
Epoch 6/50
60/60 [==============================] - 0s 1ms/step - loss: 0.1502 - accuracy: 0.9500 - val_loss

#Load pretrained DenseNet121

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'DenseNet121_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.densenet.DenseNet121(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('DenseNet121_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('DenseNet121_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('DenseNet121_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('DenseNet121_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

29089792/29084464 [==============================] - 1s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 3ms/step - loss: 14.0732 - accuracy: 0.5333 - val_loss: 26.1559 - val_accuracy: 0.5000
Epoch 2/50
60/60 [==============================] - 0s 1ms/step - loss: 6.2392 - accuracy: 0.6833 - val_loss: 18.8357 - val_accuracy: 0.5000
Epoch 3/50
60/60 [==============================] - 0s 1ms/step - loss: 4.6687 - accuracy: 0.6667 - val_loss: 7.5830 - val_accuracy: 0.5000
Epoch 4/50
60/60 [==============================] - 0s 1ms/step - loss: 2.8811 - accuracy: 0.8167 - val_loss: 4.7827 - val_accuracy: 0.5667
Epoch 5/50
60/60 [==============================] - 0s 1ms/step - loss: 1.7465 - accuracy: 0.8333 - val_loss: 4.2265 - val_accuracy: 0.4667
Epoch 6/50
60/60 [==============================] - 0s 1ms/step - loss: 0.5443 - accuracy: 0.9167 - val_los

#Load pretrained DenseNet169

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'DenseNet169_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.densenet.DenseNet169(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('DenseNet169_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('DenseNet169_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('DenseNet169_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('DenseNet169_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

51879936/51877672 [==============================] - 1s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 4ms/step - loss: 31.1693 - accuracy: 0.4500 - val_loss: 13.9196 - val_accuracy: 0.5000
Epoch 2/50
60/60 [==============================] - 0s 2ms/step - loss: 16.5808 - accuracy: 0.5167 - val_loss: 13.0544 - val_accuracy: 0.4667
Epoch 3/50
60/60 [==============================] - 0s 2ms/step - loss: 5.8742 - accuracy: 0.7167 - val_loss: 6.7656 - val_accuracy: 0.4667
Epoch 4/50
60/60 [==============================] - 0s 2ms/step - loss: 3.1778 - accuracy: 0.8167 - val_loss: 10.3795 - val_accuracy: 0.3667
Epoch 5/50
60/60 [==============================] - 0s 2ms/step - loss: 6.3147 - accuracy: 0.6833 - val_loss: 15.8024 - val_accuracy: 0.5000
Epoch 6/50
60/60 [==============================] - 0s 2ms/step - loss: 4.5178 - accuracy: 0.7667 - val_

#Load pretrained DenseNet201

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'DenseNet201_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.densenet.DenseNet201(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('DenseNet201_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('DenseNet201_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('DenseNet201_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('DenseNet201_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

74842112/74836368 [==============================] - 2s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 4ms/step - loss: 22.4886 - accuracy: 0.4333 - val_loss: 15.1740 - val_accuracy: 0.5000
Epoch 2/50
60/60 [==============================] - 0s 2ms/step - loss: 7.5998 - accuracy: 0.6500 - val_loss: 5.8308 - val_accuracy: 0.3667
Epoch 3/50
60/60 [==============================] - 0s 2ms/step - loss: 4.3003 - accuracy: 0.7167 - val_loss: 5.8871 - val_accuracy: 0.4000
Epoch 4/50
60/60 [==============================] - 0s 2ms/step - loss: 3.4077 - accuracy: 0.8000 - val_loss: 6.4184 - val_accuracy: 0.4667
Epoch 5/50
60/60 [==============================] - 0s 2ms/step - loss: 1.2795 - accuracy: 0.8833 - val_loss: 7.2053 - val_accuracy: 0.4333
Epoch 6/50
60/60 [==============================] - 0s 2ms/step - loss: 3.5495 - accuracy: 0.7500 - val_loss

#Load pretrained NASNetLarge

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 331, 331

top_model_weights_path = 'NASNetLarge_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.nasnet.NASNetLarge(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('NASNetLarge_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('NASNetLarge_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('NASNetLarge_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('NASNetLarge_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

343613440/343610240 [==============================] - 9s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 1s 24ms/step - loss: 73.4059 - accuracy: 0.4833 - val_loss: 17.7338 - val_accuracy: 0.4333
Epoch 2/50
60/60 [==============================] - 1s 22ms/step - loss: 15.1476 - accuracy: 0.7667 - val_loss: 24.9637 - val_accuracy: 0.5000
Epoch 3/50
60/60 [==============================] - 1s 22ms/step - loss: 5.2403 - accuracy: 0.9167 - val_loss: 22.7336 - val_accuracy: 0.3667
Epoch 4/50
60/60 [==============================] - 1s 22ms/step - loss: 2.5567 - accuracy: 0.8500 - val_loss: 24.7745 - val_accuracy: 0.4333
Epoch 5/50
60/60 [==============================] - 1s 22ms/step - loss: 1.6494 - accuracy: 0.9333 - val_loss: 25.5122 - val_accuracy: 0.4333
Epoch 6/50
60/60 [==============================] - 1s 22ms/step - loss: 2.4874 - accuracy: 0.96

#Load pretrained NASNETMobile

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'NASNetMobile_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.nasnet.NASNetMobile(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('NASNetMobile_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('NASNetMobile_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('NASNetMobile_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('NASNetMobile_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

#Load pretrained MobileNetV2

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'MobileNetV2_bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 60
nb_validation_samples = 30
epochs = 50
batch_size = 5


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.mobilenet_v2.MobileNetV2(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('MobileNetV2_bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('MobileNetV2_bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('MobileNetV2_bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('MobileNetV2_bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 1s 0us/step
Found 119 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Train on 60 samples, validate on 30 samples
Epoch 1/50
60/60 [==============================] - 0s 4ms/step - loss: 27.9950 - accuracy: 0.4500 - val_loss: 17.5767 - val_accuracy: 0.5000
Epoch 2/50
60/60 [==============================] - 0s 2ms/step - loss: 11.1801 - accuracy: 0.6667 - val_loss: 6.5539 - val_accuracy: 0.4667
Epoch 3/50
60/60 [==============================] - 0s 2ms/step - loss: 4.2443 - accuracy: 0.7333 - val_loss: 17.0468 - val_accuracy: 0.5000
Epoch 4/50
60/60 [==============================] - 0s 2ms/step - loss: 2.4602 - accuracy: 0.8500 - val_loss: 6.3833 - val_accuracy: 0.5000
Epoch 5/50
60/60 [==============================] - 0s 2ms/step - loss: 0.6221 - accuracy: 0.9333 - val_loss: 11.9861 - val_accuracy: 0.5333
Epoch 6/50
60/60 [==============================] - 0s 2ms/step - loss: 1.8910 - accuracy: 0.8833 - val_los

#Thank you for completing this notebook